# Лекция 4: Невронни мрежи

## От линейни модели към deep learning

**Продължителност:** 2-2.5 часа  
**Предпоставка:** Лекция 3 (Токенизация), основи на calculus и linear algebra  
**Следваща лекция:** Transformers

---
## 1. Мотивация: Защо линейните модели не достигат?

### Какво научихме досега?

В **Лекция 1** видяхме:
- Logistic regression за sentiment analysis
- Bag-of-words представяне
- Линейна граница за класификация

В **Лекция 2** видяхме:
- Word2Vec embeddings
- Семантични връзки между думи
- Плътни (dense) представяния

В **Лекция 3** видяхме:
- Byte-Pair Encoding (BPE)
- Как съвременните LLM токенизират текст

### Проблемът с линейните модели

**Линеен модел:**
$$\hat{y} = \sigma(\mathbf{w}^T\mathbf{x} + b)$$

**Ограничения:**

❌ **Не може да улови взаимодействия между features**
- "not" + "good" трябва да се третират заедно
- Линейният модел ги вижда независимо

❌ **Само една линейна граница**
- Не може да раздели сложни модели (XOR проблем)

❌ **Няма йерархия на представяния**
- Не може да научи нива на абстракция
- Low-level features → Mid-level features → High-level features

❌ **Ограничена изразителност**
- Сложни текстови модели изискват нелинейност

### Какво предлагат невронните мрежи?

✅ **Нелинейни трансформации**
- Activation functions добавят нелинейност

✅ **Йерархия на научени features**
- Слой 1: прости модели (edges, colors)
- Слой 2: комбинации (shapes, patterns)
- Слой 3: сложни концепти (objects, sentiment)

✅ **Universal approximation**
- Теоретично може да апроксимира всяка непрекъсната функция

✅ **Композиция на прости функции → сложно поведение**
- Много прости операции, стакнати заедно

### От малки мрежи към LLM

**Пътят:**
```
Прости невронни мрежи (тази лекция)
  ↓
Рекурентни мрежи (RNN) - опит, но провал
  ↓
Transformers (Лекция 5)
  ↓
Предобучени модели - BERT, GPT (Лекция 6)
  ↓
Мащабиране - милиарди параметри (Лекция 7)
  ↓
Alignment - RLHF (Лекция 8)
```

**Важно:** Всичко, което научим днес (backpropagation, optimization, training), важи и за LLM!

In [ ]:
# Импортиране на необходимите библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, make_circles
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from scipy.special import expit  # sigmoid function
import warnings
warnings.filterwarnings('ignore')

# Настройка на визуализациите
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

np.random.seed(42)

print("✓ Всички библиотеки са заредени успешно!")

### Демонстрация: XOR проблемът

**XOR (exclusive OR):**
- (0, 0) → 0
- (0, 1) → 1
- (1, 0) → 1
- (1, 1) → 0

**Проблем:** Не е линейно разделим!

In [ ]:
# Създаване на XOR dataset
X_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_xor = np.array([0, 1, 1, 0])

# Опит за обучение на линеен модел
linear_model = LogisticRegression()
linear_model.fit(X_xor, y_xor)

# Предсказания
y_pred = linear_model.predict(X_xor)
accuracy = np.mean(y_pred == y_xor)

print("XOR Проблем")
print("="*50)
print("Входове и етикети:")
for i, (x, y) in enumerate(zip(X_xor, y_xor)):
    pred = y_pred[i]
    status = "✓" if pred == y else "✗"
    print(f"  {status} x={x} → y={y}, предсказано={pred}")

print(f"\nТочност на линейния модел: {accuracy:.0%}")
print("\n💡 Линейният модел не може да реши XOR!")

In [ ]:
# Визуализация на XOR проблема
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# XOR данни
colors = ['red' if y == 0 else 'blue' for y in y_xor]
ax1.scatter(X_xor[:, 0], X_xor[:, 1], c=colors, s=200, edgecolors='black', linewidth=2)
ax1.set_xlabel('x₁', fontsize=12)
ax1.set_ylabel('x₂', fontsize=12)
ax1.set_title('XOR Проблем: Не е линейно разделим', fontsize=14, pad=15)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(-0.5, 1.5)
ax1.set_ylim(-0.5, 1.5)

# Добавяме етикети
for i, (x, y) in enumerate(zip(X_xor, y_xor)):
    ax1.annotate(f'y={y}', (x[0], x[1]), xytext=(5, 5), textcoords='offset points')

# Нелинейно разделима задача (кръгове)
X_circles, y_circles = make_circles(n_samples=200, noise=0.1, factor=0.4, random_state=42)
colors_circles = ['red' if y == 0 else 'blue' for y in y_circles]
ax2.scatter(X_circles[:, 0], X_circles[:, 1], c=colors_circles, s=30, alpha=0.6, edgecolors='black', linewidth=0.5)
ax2.set_xlabel('x₁', fontsize=12)
ax2.set_ylabel('x₂', fontsize=12)
ax2.set_title('Концентрични кръгове: Също нелинейна задача', fontsize=14, pad=15)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 Червените и сините точки не могат да бъдат разделени с права линия!")
print("   Необходима е нелинейна граница → Невронни мрежи!")

---
## 2. Градивни елементи: Неврони и слоеве

### 2.1 Единичен неврон

**Математически модел:**

1. **Линейна комбинация:**
$$z = \sum_{i=1}^{n} w_i x_i + b = \mathbf{w}^T\mathbf{x} + b$$

2. **Активация:**
$$a = f(z)$$

Където:
- $\mathbf{x}$ = входен вектор
- $\mathbf{w}$ = тегла (weights)
- $b$ = bias
- $f$ = activation function
- $a$ = изходна активация

**Биологична инспирация:**
- Неврон в мозъка получава сигнали от други неврони
- Ако сумата на сигналите надхвърли праг → неврон "се активира"
- Изкуственият неврон е **много опростен** модел

### 2.2 Активационни функции

**Защо са необходими?**

Без активация:
$$f(W_2 \cdot f(W_1 \cdot x)) = f(W_2 \cdot W_1 \cdot x) = f(W \cdot x)$$

Мрежата се колапсира до линеен модел!

**Популярни активационни функции:**

#### 1. Sigmoid
$$\sigma(z) = \frac{1}{1 + e^{-z}}$$

- **Изход:** (0, 1)
- **Използване:** Output layer за binary classification
- **Проблем:** Vanishing gradients

#### 2. Tanh
$$\tanh(z) = \frac{e^z - e^{-z}}{e^z + e^{-z}}$$

- **Изход:** (-1, 1)
- **Предимство:** Zero-centered (по-добре от sigmoid)
- **Проблем:** Също vanishing gradients

#### 3. ReLU (Rectified Linear Unit)
$$\text{ReLU}(z) = \max(0, z)$$

- **Изход:** [0, ∞)
- **Предимства:**
  - Решава vanishing gradient проблема
  - Бързо изчисление
  - Най-популярна в съвременните мрежи
- **Проблем:** "Dead ReLU" (неврони могат да умрат)

#### 4. GELU (Gaussian Error Linear Unit)
$$\text{GELU}(z) \approx z \cdot \sigma(1.702z)$$

- **Използва се в:** GPT-2, GPT-3, BERT
- **Предимства:** Smooth approximation на ReLU
- State-of-the-art за Transformers

In [ ]:
# Дефиниране на активационни функции
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def tanh(z):
    return np.tanh(z)

def relu(z):
    return np.maximum(0, z)

def gelu(z):
    # GELU approximation
    return 0.5 * z * (1 + np.tanh(np.sqrt(2 / np.pi) * (z + 0.044715 * z**3)))

# Производни (derivatives)
def sigmoid_derivative(z):
    s = sigmoid(z)
    return s * (1 - s)

def tanh_derivative(z):
    return 1 - np.tanh(z)**2

def relu_derivative(z):
    return (z > 0).astype(float)

print("✓ Активационни функции дефинирани!")

In [ ]:
# Визуализация на активационни функции
z = np.linspace(-5, 5, 200)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Sigmoid
axes[0, 0].plot(z, sigmoid(z), 'b-', linewidth=2, label='sigmoid(z)')
axes[0, 0].plot(z, sigmoid_derivative(z), 'r--', linewidth=2, label="sigmoid'(z)")
axes[0, 0].axhline(y=0, color='k', linewidth=0.5)
axes[0, 0].axvline(x=0, color='k', linewidth=0.5)
axes[0, 0].set_title('Sigmoid', fontsize=14, pad=10)
axes[0, 0].set_xlabel('z')
axes[0, 0].set_ylabel('Activation')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].text(0.05, 0.95, 'Изход: (0, 1)\nVanishing gradients', 
                transform=axes[0, 0].transAxes, fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Tanh
axes[0, 1].plot(z, tanh(z), 'b-', linewidth=2, label='tanh(z)')
axes[0, 1].plot(z, tanh_derivative(z), 'r--', linewidth=2, label="tanh'(z)")
axes[0, 1].axhline(y=0, color='k', linewidth=0.5)
axes[0, 1].axvline(x=0, color='k', linewidth=0.5)
axes[0, 1].set_title('Tanh', fontsize=14, pad=10)
axes[0, 1].set_xlabel('z')
axes[0, 1].set_ylabel('Activation')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].text(0.05, 0.95, 'Изход: (-1, 1)\nZero-centered', 
                transform=axes[0, 1].transAxes, fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# ReLU
axes[1, 0].plot(z, relu(z), 'b-', linewidth=2, label='ReLU(z)')
axes[1, 0].plot(z, relu_derivative(z), 'r--', linewidth=2, label="ReLU'(z)")
axes[1, 0].axhline(y=0, color='k', linewidth=0.5)
axes[1, 0].axvline(x=0, color='k', linewidth=0.5)
axes[1, 0].set_title('ReLU', fontsize=14, pad=10)
axes[1, 0].set_xlabel('z')
axes[1, 0].set_ylabel('Activation')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].text(0.05, 0.95, 'Изход: [0, ∞)\nНай-популярна!', 
                transform=axes[1, 0].transAxes, fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))

# GELU
axes[1, 1].plot(z, gelu(z), 'b-', linewidth=2, label='GELU(z)')
axes[1, 1].plot(z, relu(z), 'g--', linewidth=1, alpha=0.5, label='ReLU (за сравнение)')
axes[1, 1].axhline(y=0, color='k', linewidth=0.5)
axes[1, 1].axvline(x=0, color='k', linewidth=0.5)
axes[1, 1].set_title('GELU', fontsize=14, pad=10)
axes[1, 1].set_xlabel('z')
axes[1, 1].set_ylabel('Activation')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].text(0.05, 0.95, 'Smooth ReLU\nИзползва се в GPT!', 
                transform=axes[1, 1].transAxes, fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))

plt.tight_layout()
plt.show()

print("\n💡 Наблюдения:")
print("   • Sigmoid и Tanh: производната клони към 0 при големи |z| → vanishing gradients")
print("   • ReLU: производната е константа (1) за z > 0 → няма vanishing gradients")
print("   • GELU: smooth версия на ReLU, използва се в съвременните LLM")

### 2.3 Слоеве и архитектура

**Многослойна невронна мрежа:**

```
Input Layer     Hidden Layer 1    Hidden Layer 2    Output Layer
    x₁              h₁⁽¹⁾             h₁⁽²⁾              ŷ
    x₂     →        h₂⁽¹⁾      →      h₂⁽²⁾       →   
    x₃              h₃⁽¹⁾             h₃⁽²⁾            
    x₄              h₄⁽¹⁾                              
```

**Терминология:**
- **Input layer:** Получава входните данни (embeddings за NLP)
- **Hidden layers:** Научени представяния (може да има много)
- **Output layer:** Финални предсказания (task-specific)

**Fully Connected (Dense) Layer:**
- Всеки неврон се свързва с **всички** неврони от предишния слой
- Параметри: $W^{[l]} \in \mathbb{R}^{n_l \times n_{l-1}}$ и $b^{[l]} \in \mathbb{R}^{n_l}$

**Нотация:**
- Слой $l$ има $n_l$ неврона
- $W^{[l]}$ = матрица с тегла
- $b^{[l]}$ = bias вектор
- $a^{[l]}$ = активации на слой $l$
- $z^{[l]}$ = линейна комбинация преди активация

### Пример: Sentiment Classification мрежа

**Задача:** Класифициране на филмови ревюта (положително/отрицателно)

**Архитектура:**
```
Input: Averaged Word2Vec (300 dimensions)
  ↓
Dense Layer 1: 300 → 128 (ReLU)
  ↓
Dense Layer 2: 128 → 64 (ReLU)
  ↓
Output Layer: 64 → 1 (Sigmoid)
```

**Параметри:**
- Layer 1: $300 \times 128 + 128 = 38,528$
- Layer 2: $128 \times 64 + 64 = 8,256$
- Output: $64 \times 1 + 1 = 65$
- **Общо:** 46,849 параметри

In [ ]:
# Визуализация на архитектура на мрежа
def visualize_network_architecture():
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Позиции на слоевете
    layers = [4, 6, 4, 1]  # Брой неврони във всеки слой
    layer_names = ['Input\nLayer', 'Hidden\nLayer 1', 'Hidden\nLayer 2', 'Output\nLayer']
    layer_positions = [1, 3, 5, 7]
    
    # Рисуваме невроните
    neuron_positions = {}
    for layer_idx, (n_neurons, x_pos) in enumerate(zip(layers, layer_positions)):
        y_positions = np.linspace(1, 8, n_neurons)
        neuron_positions[layer_idx] = []
        for y_pos in y_positions:
            circle = plt.Circle((x_pos, y_pos), 0.2, color='lightblue', ec='black', linewidth=2)
            ax.add_patch(circle)
            neuron_positions[layer_idx].append((x_pos, y_pos))
    
    # Рисуваме връзките (само няколко за яснота)
    for layer_idx in range(len(layers) - 1):
        for i, (x1, y1) in enumerate(neuron_positions[layer_idx]):
            for j, (x2, y2) in enumerate(neuron_positions[layer_idx + 1]):
                # Рисуваме само част от връзките за яснота
                if i % 2 == 0 or j % 2 == 0:
                    ax.plot([x1, x2], [y1, y2], 'gray', alpha=0.3, linewidth=0.5)
    
    # Добавяме етикети
    for i, (name, x_pos) in enumerate(zip(layer_names, layer_positions)):
        ax.text(x_pos, 0.3, name, ha='center', fontsize=12, fontweight='bold')
        
        # Добавяме информация за размерите
        if i == 0:
            ax.text(x_pos, -0.3, '300d', ha='center', fontsize=10, style='italic')
        elif i == 1:
            ax.text(x_pos, -0.3, '128 neurons', ha='center', fontsize=10, style='italic')
        elif i == 2:
            ax.text(x_pos, -0.3, '64 neurons', ha='center', fontsize=10, style='italic')
        else:
            ax.text(x_pos, -0.3, 'probability', ha='center', fontsize=10, style='italic')
    
    # Добавяме активационни функции
    ax.text(2, 9, 'ReLU', ha='center', fontsize=11, bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
    ax.text(4, 9, 'ReLU', ha='center', fontsize=11, bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
    ax.text(6, 9, 'Sigmoid', ha='center', fontsize=11, bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.7))
    
    ax.set_xlim(0, 8)
    ax.set_ylim(-1, 9.5)
    ax.axis('off')
    ax.set_title('Sentiment Classification Neural Network Architecture', fontsize=16, pad=20, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

visualize_network_architecture()

print("💡 Всяка линия представлява един параметър (weight).")
print("   Fully connected означава: всеки неврон е свързан с всички неврони от предишния слой.")